### Module importation

In [1]:
import pandas as pd
import glob
import os
from pathlib import Path
import chardet
import datetime as dt
import numpy as np

### Encoding verification

In [2]:
csv_dir = "data_stations/raw_data/*"
files = glob.glob(csv_dir)
file = files[0] # exemplo de arquivo, só para identificação do padrão de encoding

def encoding_detection(file):
    with open(file, 'rb') as f:
        result = chardet.detect(f.read())
        return result['encoding']

encodig = encoding_detection(file)

print(f'Encoding: {encodig}')


Encoding: ISO-8859-1


### Preprocessing and data insertion on database

In [3]:
csv_dir = "data_stations/raw_data/*"
files = glob.glob(csv_dir)

# Criação de uma pasta que receberá os arquivos csv configurados/pre-processados
destiny_folder = "data_stations/preprocessed_data"      
if not os.path.exists(destiny_folder):
    os.makedirs(destiny_folder)

cols_to_use = list(range(19))
rows_to_skip = list(range(9))

datasets = []

df_concat = pd.DataFrame()


for idx, file in enumerate(files):
    
    file_name = file.split("\\")[-1]
    
    data = pd.read_csv(file, encoding=encodig, sep=";",
                    skiprows=rows_to_skip , header=None, usecols=cols_to_use)

    cols_renamed = "Data;Hora UTC;PRECIPITACAO TOTAL (mm);PRESSAO ATMOSFERICA (mB);PRESSAO ATMOSFERICA MAX. (mB);PRESSAO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB);RADIACAO GLOBAL (Kj/m2);TEMPERATURA DO AR - BULBO SECO(C);TEMPERATURA DO PONTO DE ORVALHO (C);TEMPERATURA MAXIMA (C);TEMPERATURA MINIMA (C);TEMPERATURA ORVALHO MAX. (C);TEMPERATURA ORVALHO MIN. (C);UMIDADE REL. MAX. (%);UMIDADE REL. MIN. (%);UMIDADE RELATIVA DO AR (%);VENTO, DIRECAO (gr);VENTO, RAJADA MAX. (m/s);VENTO VELOCIDADE (m/s)]"      
    cols_renamed = cols_renamed.lower().split(";")

    data.columns = cols_renamed

    # conversão das células que foram lidas como não numéricas para numéricas
    cols_to_convert = [x for x in cols_renamed if x not in ['data', 'hora utc']]
    
    for col in cols_to_convert:
        data[col] = pd.to_numeric(data[col], errors='coerce')
        
    # substituição dos valores inválidos (-9999) por NaN
    data = data.replace(-9999, np.nan)

    # Conversão para timestamp
    # não esquecer que todas os nomes de colunas havia sido convertidas para minusculo
    try:
        data["timestamp"] = data["data"] + ' ' + data["hora utc"] + ':00'
        data['timestamp'] = data['timestamp'].apply(lambda x: dt.datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S'))
        
    except ValueError:
        data["timestamp"] = data["data"] + ' ' + data["hora utc"].str[0:2] + ':00:00'
        data['timestamp'] = data['timestamp'].apply(lambda x: dt.datetime.strptime(str(x), '%Y/%m/%d %H:%M:%S'))
        
    data = data.drop(["data", "hora utc"], axis=1) #eliminação das colunas data e hora, que não serão mais necessárias com o timestamp
    
    # definindo uma coluna que guarda o nome da estação
    station= file_name.split("_")[4]
    data["nome_estacao"] = station

    # Importante! Como os nomes das estações podem se repetir, é bm armazenar o ID de cada uma
    id_station= file_name.split("_")[3]
    data["id_estacao"] = id_station
    
    # reorganizando as colunas de maneira que o nome da estação e a data apareceçam antes das medidas
    ordered_cols = ['id_estacao', 'nome_estacao', 'timestamp'] + [x for x in cols_renamed if x not in ['data', 'hora utc']]
    data = data[ordered_cols]
    
    datasets.append(data)
        
    # validação, indicando quais colunas restaram e seus tipos, pós preprocessadas
    if idx == 0:
        print("Colunas do dataset:\n")
        print(data.dtypes)
        
df_concat = pd.concat(datasets, ignore_index=True)

nan_count = df_concat.isna().sum()
print(f"\nQuantidade dados NaN: {nan_count}")

Colunas do dataset:

id_estacao                                                  object
nome_estacao                                                object
timestamp                                           datetime64[ns]
precipitacao total (mm)                                    float64
pressao atmosferica (mb)                                   float64
pressao atmosferica max. (mb)                              float64
pressao atmosferica min. na hora ant. (aut) (mb)           float64
radiacao global (kj/m2)                                    float64
temperatura do ar - bulbo seco(c)                          float64
temperatura do ponto de orvalho (c)                        float64
temperatura maxima (c)                                     float64
temperatura minima (c)                                     float64
temperatura orvalho max. (c)                               float64
temperatura orvalho min. (c)                               float64
umidade rel. max. (%)                    

### Complete dataset saving

In [4]:
dataset_name = "all_stations.csv"
    
path_to_csv = os.path.join(destiny_folder, dataset_name)
df_concat.to_csv(path_to_csv, index=False)